In [1]:
import pandas as pd
import re
import emoji
import string
import nltk
from bs4 import BeautifulSoup
from autocorrect import Speller
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

# Initialize tools
spell = Speller(lang='en')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Dictionary of slang words and their replacements
slang_dict = {
    "tbh": "to be honest",
    "omg": "oh my god",
    "lol": "laugh out loud",
    "idk": "I don't know",
    "brb": "be right back",
    "btw": "by the way",
    "imo": "in my opinion",
    "smh": "shaking my head",
    "fyi": "for your information",
    "np": "no problem",
    "ikr": "I know right",
    "asap": "as soon as possible",
    "bff": "best friend forever",
    "gg": "good game",
    "hmu": "hit me up",
    "rofl": "rolling on the floor laughing"
}

# Contractions dictionary
contractions_dict = {
    "wasn't": "was not",
    "isn't": "is not",
    "aren't": "are not",
    "weren't": "were not",
    "doesn't": "does not",
    "don't": "do not",
    "didn't": "did not",
    "can't": "cannot",
    "couldn't": "could not",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "won't": "will not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "i'm": "i am",
    "you're": "you are",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "we're": "we are",
    "they're": "they are",
    "i've": "i have",
    "you've": "you have",
    "we've": "we have",
    "they've": "they have",
    "i'd": "i would",
    "you'd": "you would",
    "he'd": "he would",
    "she'd": "she would",
    "we'd": "we would",
    "they'd": "they would",
    "i'll": "i will",
    "you'll": "you will",
    "he'll": "he will",
    "she'll": "she will",
    "we'll": "we will",
    "they'll": "they will",
    "let's": "let us",
    "that's": "that is",
    "who's": "who is",
    "what's": "what is",
    "where's": "where is",
    "when's": "when is",
    "why's": "why is"
}

# Remove any URLs that start with "http" or "www" from the text
def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text)

# Extracts only the text, removing all HTML tags
def remove_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Replace emoji with ''
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

# Replace internet slang/chat words
def replace_slang(text):
    escaped_slang_words = []
    for word in slang_dict.keys():
        escaped_word = re.escape(word)
        escaped_slang_words.append(escaped_word)
    slang_pattern = r'\b(' + '|'.join(escaped_slang_words) + r')\b'
    def replace_match(match):
        slang_word = match.group(0)
        return slang_dict[slang_word.lower()]
    replaced_text = re.sub(slang_pattern, replace_match, text, flags=re.IGNORECASE)
    return replaced_text

# Build the regex pattern for contractions
escaped_contractions = []
for contraction in contractions_dict.keys():
    escaped_contraction = re.escape(contraction)
    escaped_contractions.append(escaped_contraction)
joined_contractions = "|".join(escaped_contractions)
contractions_pattern = r'\b(' + joined_contractions + r')\b'
compiled_pattern = re.compile(contractions_pattern, flags=re.IGNORECASE)

# Function to replace contractions
def replace_contractions(text):
    def replace_match(match):
        matched_word = match.group(0)
        lower_matched_word = matched_word.lower()
        expanded_form = contractions_dict[lower_matched_word]
        return expanded_form
    expanded_text = compiled_pattern.sub(replace_match, text)
    return expanded_text

# Function to remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Function to remove numbers
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# Function to correct spelling
def correct_spelling(text):
    return spell(text)

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

# Function to map NLTK POS tags to WordNet POS tags
def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Function to lemmatize text with POS tagging
def lemmatize_text(text):
    if not isinstance(text, str):
        return ""
    words = word_tokenize(text)
    pos_tags = pos_tag(words)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return " ".join(lemmatized_words)

# Function to tokenize text
def tokenize_text(text):
    if not isinstance(text, str):
        return []
    return word_tokenize(text)

# Function to apply all preprocessing steps
def preprocess_text(text):
    text = text.lower()                  # Step 1: Lowercasing
    text = remove_urls(text)             # Step 2: Remove URLs
    text = remove_html(text)             # Step 3: Remove HTML tags
    text = remove_emojis(text)           # Step 4: Remove Emojis
    text = replace_slang(text)           # Step 5: Replace Slang
    text = replace_contractions(text)    # Step 6: Expand Contractions
    text = remove_punctuation(text)      # Step 7: Remove Punctuation
    text = remove_numbers(text)          # Step 8: Remove Numbers
    text = correct_spelling(text)        # Step 9: Correct Spelling
    text = remove_stopwords(text)        # Step 10: Remove Stopwords
    text = lemmatize_text(text)          # Step 11: Lemmatization
    text = tokenize_text(text)           # Step 12: Tokenization
    return text

# Load dataset
df = pd.read_csv("Review.csv")

# Apply preprocessing pipeline
df["processed"] = df["Review"].apply(preprocess_text)

# Save the cleaned dataset
df.to_csv("Processed_Reviews2.csv", index=False)

# Display the first few rows
print(df[["Review", "processed"]].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 662: invalid start byte

In [2]:
import pandas as pd
import re
import emoji
import string
import nltk
from bs4 import BeautifulSoup
from autocorrect import Speller
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

# Initialize tools
spell = Speller(lang='en')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Slang dictionary
slang_dict = {
    "tbh": "to be honest",
    "omg": "oh my god",
    "lol": "laugh out loud",
    "idk": "I don't know",
    "brb": "be right back",
    "btw": "by the way",
    "imo": "in my opinion",
    "smh": "shaking my head",
    "fyi": "for your information",
    "np": "no problem",
    "ikr": "I know right",
    "asap": "as soon as possible",
    "bff": "best friend forever",
    "gg": "good game",
    "hmu": "hit me up",
    "rofl": "rolling on the floor laughing"
}

# Contractions dictionary
contractions_dict = {
    "wasn't": "was not", "isn't": "is not", "aren't": "are not",
    "weren't": "were not", "doesn't": "does not", "don't": "do not",
    "didn't": "did not", "can't": "cannot", "couldn't": "could not",
    "shouldn't": "should not", "wouldn't": "would not", "won't": "will not",
    "haven't": "have not", "hasn't": "has not", "hadn't": "had not",
    "i'm": "i am", "you're": "you are", "he's": "he is", "she's": "she is",
    "it's": "it is", "we're": "we are", "they're": "they are",
    "i've": "i have", "you've": "you have", "we've": "we have",
    "they've": "they have", "i'd": "i would", "you'd": "you would",
    "he'd": "he would", "she'd": "she would", "we'd": "we would",
    "they'd": "they would", "i'll": "i will", "you'll": "you will",
    "he'll": "he will", "she'll": "she will", "we'll": "we will",
    "they'll": "they will", "let's": "let us", "that's": "that is",
    "who's": "who is", "what's": "what is", "where's": "where is",
    "when's": "when is", "why's": "why is"
}

# Build contractions pattern
escaped_contractions = [re.escape(c) for c in contractions_dict.keys()]
compiled_pattern = re.compile(r'\b(' + '|'.join(escaped_contractions) + r')\b', flags=re.IGNORECASE)

# Preprocessing functions
def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text)

def remove_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

def replace_slang(text):
    escaped = [re.escape(w) for w in slang_dict.keys()]
    pattern = r'\b(' + '|'.join(escaped) + r')\b'
    return re.sub(pattern, lambda m: slang_dict[m.group(0).lower()], text, flags=re.IGNORECASE)

def replace_contractions(text):
    return compiled_pattern.sub(lambda m: contractions_dict[m.group(0).lower()], text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

def correct_spelling(text):
    return spell(text)

def remove_stopwords(text):
    words = text.split()
    return " ".join([word for word in words if word.lower() not in stop_words])

def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'): return wordnet.ADJ
    elif nltk_tag.startswith('V'): return wordnet.VERB
    elif nltk_tag.startswith('N'): return wordnet.NOUN
    elif nltk_tag.startswith('R'): return wordnet.ADV
    else: return wordnet.NOUN

def lemmatize_text(text):
    if not isinstance(text, str): return ""
    words = word_tokenize(text)
    pos_tags = pos_tag(words)
    return " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags])

def tokenize_text(text):
    if not isinstance(text, str): return []
    return word_tokenize(text)

# Full preprocessing pipeline
def preprocess_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = remove_urls(text)
    text = remove_html(text)
    text = remove_emojis(text)
    text = replace_slang(text)
    text = replace_contractions(text)
    text = remove_punctuation(text)
    text = remove_numbers(text)
    text = correct_spelling(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    text = tokenize_text(text)
    return text

# Load dataset with latin-1 encoding to handle special characters
df = pd.read_csv("Review.csv", encoding='latin-1')

# Apply preprocessing pipeline
df["processed"] = df["Review"].apply(preprocess_text)

# Save the cleaned dataset
df.to_csv("Processed_Reviews2.csv", index=False)

# Display the first few rows
print(df[["Review", "processed"]].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/9bd7767f-e2b1-4f3e-a47c-
[nltk_data]     80a0c1669bd6/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                              Review  \
0  The product arrived on time. Packaging was gre...   
1           THIS PRODUCT IS JUST AMAZING! I LOVE IT.   
2  I bought this phone for $799, and it has a 120...   
3  Wow!!! This product is awesome... but a bit ex...   
4                The laptop works perfectly fine.      

                                           processed  
0  [product, arrive, time, packaging, great, qual...  
1                             [product, amaze, love]  
2          [buy, phone, hz, display, totally, worth]  
3            [wow, product, awesome, bit, expensive]  
4                    [laptop, work, perfectly, fine]  
